In [ ]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
endpoint = os.getenv("OPENAI_ENDPOINT")
client = OpenAI(
            api_key=api_key,
            base_url=endpoint
        )

## **Agent Classes**

### Memory Class
- Memory class that can handle more complicated cases
- Mainly three methods, add_message, get_messages, and get_last_message

In [ ]:
class Memory:
    def __init__(self, verbose: bool = False):
        self._messages: List[Dict[str, str]] = []
        self.verbose = verbose
    
    def add_message(self, role: Literal['user', 'system', 'assistant'], content: str):
        self._messages.append({
            "role": role,
            "content": content
        })
        if self.verbose:
            self._log_last_message()

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def _log_last_message(self):
        print(f"### {self.last_message()['role']} message ###\n".upper())
        print(f"{self.last_message()['content']} \n")
        print("\n________________________________________________________________\n")

### The Agent Class (adding self-reflection)
This is a foundational structure of an AI Agent that can process user inputs and generate responses using a language model. 

**Objective**
- Be initialized with configurable settings, including a name, role, instructions, and model parameters.
- Send user messages to a language model, ensuring that the response aligns with the given role and instructions.
- Return the AI-generated response as a string.

**Steps**
- Create a constructor that allows customization of the agent’s name, role, and instructions.
- Ensure the agent interacts with a language model, passing the user message along with system instructions.
- Implement a method to handle message processing, ensuring the response is retrieved correctly.

**Considerations**
- The role and instructions should guide the agent’s behavior in generating responses.
- The model's parameters (like temperature) should be configurable.
- Ensure the agent maintains simplicity while providing a structured response.


In [ ]:

class Agent():
    """A simple AI Agent"""

    SELF_CRITIQUE_PROMPT = """
    Reflect on your previous response...
    Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. 
    Provide a revised response if necessary in a Json Output structure:
    {
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
    }
    """

    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        verbose: bool = False,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.verbose = verbose

        self.client = client

        self.memory = Memory(self.verbose)
        self.memory.add_message(role="system", content=f"You're an AI Agent, your role is {self.role}, "f"and you need to {self.instructions}")
        
        
    def invoke(self, 
               user_message: str, 
               self_reflection: bool = False, 
               max_iter: int = 1) -> str:

        if not self_reflection:
            max_iter = 0
        elif max_iter < 1:
            max_iter = 1
        elif max_iter > 3:
            max_iter = 3

        assistant_messsage = self._get_completion(user_message)

        iter_count = 0
        while iter_count < max_iter:
            assistant_messsage = self._get_completion(Agent.SELF_CRITIQUE_PROMPT)
            iter_count = iter_count +1

        return assistant_messsage.content

    def _get_completion(self, message:str) -> ChatCompletionMessage:
        self.memory.add_message(role="user", content=message)
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=self.memory.get_messages()
        )
        self.memory.add_message(role="assistant", content=response.choices[0].message.content)
        
        return response.choices[0].message





## Building Some Agents

In [ ]:
agent = Agent(verbose=True)
agent.invoke(
    user_message="Pick only one. Who is the best character in Game of Thrones?",
    self_reflection=True,
)

In [ ]:
json.loads(agent.memory.last_message()["content"])["updated_response"]